# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [125]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/phungnguyen/Documents/Data_Engineer_DA/Azure/Udacity_Azure Data Engineer_20251102~/Project1_FAQs_DataModelWithCassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
    # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Get total number of rows 
#print(len(full_data_rows_list))
# Check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [128]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# Make a connection to a Cassandra instance your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [130]:
# Create a Keyspace 
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

#### Set Keyspace

In [131]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [ ]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## **Primary Key Explanation:**
## Partition Key: sessionId → ensures all items of a session are stored together
## Clustering Column: itemInSession → orders the items within the session
## This design allows us to efficiently query artist, song, and length for a specific session and itemInSession without using ALLOW FILTERING.

query1 = "CREATE TABLE IF NOT EXISTS music_app_history "
query1 = query1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)                 

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query1 = "INSERT INTO music_app_history(sessionId, itemInSession,artist,song,length)"
        query1 = query1 + "VALUES(%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9],float(line[5])))

In [137]:
## Add in the SELECT statement to verify the data was entered into the table
query1 = """
SELECT artist, song, length 
FROM music_app_history 
WHERE sessionId = 338 AND itemInSession = 4
"""
rows = session.execute(query1)
df = pd.DataFrame(rows)
print(df)

      artist                             song      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [ ]:
##**Primary Key Explanation:**
## Partition Key: (userId, sessionId) → groups all items of a user within a session together
## Clustering Column: itemInSession → ensures the items are sorted in the order they were played
## This design allows us to retrieve all songs for a user in a session in order without using ALLOW FILTERING.

query2= "CREATE TABLE IF NOT EXISTS artist_song_user"
query2 = query2 + "(userId int, sessionId int,itemInSession int, artist text, song text,firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ##Assign the INSERT statements into the `query` variable
        query2 = "INSERT INTO artist_song_user(userId, sessionId,itemInSession, artist, song,firstName,lastName)"
        query2 = query2 + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        ##Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0],line[9],line[1],line[4]))                   

In [143]:
## Add in the SELECT statement to verify the data was entered into the table
query2 = """
SELECT artist,song,firstName, lastName
FROM artist_song_user 
WHERE userid = 10 AND sessionid = 182
"""
rows = session.execute(query2)
df = pd.DataFrame(rows)
print(df)

              artist                                               song  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


   

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
## **Primary Key Explanation:**
## Partition Key: song → groups all users who listened to the same song
## Clustering Column: userId → sorts the users within the song partition
## This design allows efficient lookup of all users who listened to a specific song without needing ALLOW FILTERING.

query3 = """
CREATE TABLE IF NOT EXISTS song_user_history (
    song text,
    userId int,
    firstName text,
    lastName text,
    PRIMARY KEY (song, userId)
)
"""
try:
    session.execute(query3)
except Exception as e:
    print(e)

#INSERT
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query3 = "INSERT INTO song_user_history (song, userId, firstName, lastName)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))                  

In [146]:
query3 = "SELECT firstName, lastName FROM song_user_history WHERE song='All Hands Against His Own'"
rows = session.execute(query3)
df = pd.DataFrame(rows)
print(df)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [ ]:
for t in ["music_app_history", "artist_song_user", "song_user_history"]:
    query = f"DROP TABLE {t}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)


### Close the session and cluster connection¶

In [124]:
session.shutdown()
cluster.shutdown()